In [ ]:
import pickle
import nltk
import json
from tqdm import tqdm

In [ ]:
# MAX_DISH_COUNT = 100

with open('./categories/Chinese.pkl', 'rb') as f:
    cuisine_reviews = pickle.load(f)    
with open('ChineseDishes.txt', 'r') as f:
    dishes = {line.replace("\n", "").lower() : {} for line in f.readlines()}
#     dishes = {line.replace("\n", "").lower() : {} for line in f.readlines()[0:MAX_DISH_COUNT]}

for _review in tqdm(cuisine_reviews):
    _review['sentences'] = nltk.sent_tokenize(_review['text'])
    
rest_id2name = {}
path2files="../yelp_dataset_challenge_academic_dataset/"
path2buisness=path2files+"yelp_academic_dataset_business.json"

with open (path2buisness, 'r') as f:
    for line in f.readlines():
        business_json = json.loads(line)
        if (business_json['business_id'] in rest_id2name):
            if rest_id2name[business_json['business_id']] != business_json['name']:
                print('dup name found!')
        else: 
            rest_id2name[business_json['business_id']] = business_json['name']

In [ ]:
%%time
from gensim.summarization.summarizer import summarize

APP_FOLDER = "../app/data/processed/"
path2files = "../yelp_dataset_challenge_academic_dataset/"
path2buisness = path2files+"yelp_academic_dataset_business.json"

biz = {}
with open (path2buisness, 'r') as f:
    for line in f.readlines():
        business_json = json.loads(line)
        biz[business_json['business_id']] = {}
        for _, key in enumerate(business_json):
            if not key in ['business_id', 'type']:
                biz[business_json['business_id']][key] = business_json[key]

biz2info = {}
for _review in cuisine_reviews:
    if not (_review['business_id'] in biz2info):
        biz2info[_review['business_id']] = {}
        for _, key in enumerate(biz[_review['business_id']]):
            biz2info[_review['business_id']][key] = biz[_review['business_id']][key]
        biz2info[_review['business_id']]['review_summary'] = ''
        biz2info[_review['business_id']]['reviews'] = []
    
    biz2info[_review['business_id']]['reviews'].append({'review_id': _review['review_id'],
                                               'user_id': _review['user_id'], 
                                               'stars': _review['stars'],
                                               'date': _review['date'],
                                               'text': _review['text']
                                              })
    biz2info[_review['business_id']]['review_summary'] += "." + _review['text']

for _id in tqdm(biz2info):
     biz2info[_id]['review_summary'] = summarize(biz2info[_id]['review_summary'])

with open (APP_FOLDER + 'biz2review.plk', 'wb') as f:
    pickle.dump(biz2info, f)

In [ ]:
for _id in tqdm(biz2info):
    print(biz2info[_id]['review_summary'])

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

for _dish in tqdm(dishes):
    dishes[_dish] = {'rest2review_count':{}, 'review2polarity':{}, 'rest2polarity':{}, 'rest2score':{}}
    _dish2rest = set()
    _review_count = 0
    _sum_review_polarity = 0
    _sum_rest_polarity = 0
    
    for _review in cuisine_reviews:
        _rest_id = _review['business_id']
        _review_id = _review['review_id']
#         _rest_name = rest_id2name[_business_id]
        _review_polarity = 0
        _review_sentence_count = 0
        for _sentence in _review['sentences']:
            _count = _sentence.count(_dish)
            if _count > 0:
                _review_sentence_count += 1
                _ss = sia.polarity_scores(_sentence)
                _score = (_ss['pos'] + 1e-10)/ (_ss['neg'] + 1e-10) * 0.5
                if _score <= 1:
                    _review_polarity += _score
                else:
                    _review_polarity = 1
        
        if _review_sentence_count > 0:
            _review_polarity = _review_polarity / _review_sentence_count
            _sum_review_polarity += _review_polarity
            _review_count += 1
            dishes[_dish]['review2polarity'][_review_id] = _review_polarity
            if _rest_id in dishes[_dish]['rest2review_count']:
                dishes[_dish]['rest2review_count'][_rest_id] += 1
                dishes[_dish]['rest2polarity'][_rest_id] += _review_polarity
            else:
                dishes[_dish]['rest2review_count'][_rest_id] = 1
                dishes[_dish]['rest2polarity'][_rest_id] = _review_polarity
            
            
    for _id in dishes[_dish]['rest2polarity']:
        dishes[_dish]['rest2score'][_id] = dishes[_dish]['rest2polarity'][_id]
        dishes[_dish]['rest2polarity'][_id] /= dishes[_dish]['rest2review_count'][_id]
        _sum_rest_polarity += dishes[_dish]['rest2polarity'][_id]
    
    dishes[_dish]['rest2polarity'] = sorted(dishes[_dish]['rest2polarity'].items(), key=lambda kv: kv[1], reverse = True)
    dishes[_dish]['rest2review_count'] = sorted(dishes[_dish]['rest2review_count'].items(), key=lambda kv: kv[1], reverse = True)
    dishes[_dish]['rest2score'] = sorted(dishes[_dish]['rest2score'].items(), key=lambda kv: kv[1], reverse = True)
    dishes[_dish]['review_count'] = len(dishes[_dish]['review2polarity'])
    dishes[_dish]['rest_count'] = len(dishes[_dish]['rest2review_count'])
    dishes[_dish]['rest_polarity'] = _sum_rest_polarity / dishes[_dish]['rest_count']
    dishes[_dish]['review_polarity'] = _sum_review_polarity / dishes[_dish]['review_count']

In [ ]:
sorted_rest_dishes = sorted(dishes.items(), key=lambda kv: kv[1]['rest_count'], reverse=True)
sorted_review_dishes = sorted(dishes.items(), key=lambda kv: kv[1]['review_count'], reverse=True)

with open (APP_FOLDER + 'dish2rest.plk', 'wb') as f:
    pickle.dump(sorted_rest_dishes, f)

# dish_popularity = [(key, dishes[key]['rest_count']) for _, key in enumerate(dishes)]
# dish_popularity = sorted(dish_popularity, key = lambda kv: kv[1], reverse = True)

# with open (APP_FOLDER + 'dish2pop.plk', 'wb') as f:
#     pickle.dump(dish_popularity, f)